In [1]:
import itertools

In [2]:
'''
Transformation rules:
set       binary     index
{1}       001        1
{2}       010        2
{3}       100        4
{1,2}     011        3
{1,3}     101        5
{2,3}     110        6
{1,2,3}   111        7
'''
class dataset:
    def __init__(self,N,K):
        self.N = N
        self.K = K
        
        self.vfunction_for_N = [-1] * 2**N #v() for set N
        self.vfunction_for_K = [-1] * 2**N #v() for set K
    
    #transform the index of array into a set
    def index_into_set(self,index):
        set = ()
        for i in range(0,self.N):
            if index & (1 << i) > 0:
                set = set + (i+1,)
        return set

    #transform the set into the index of array
    def set_into_index(self,set):
        index = 0
        for i in set:
            index = index + 2**(i-1)
        return index

    def generate_dataset(self,characteristic_function):
        for i in characteristic_function.keys():
            self.vfunction_for_N[self.set_into_index(i)] = characteristic_function[i]
            if(len(i) <= self.K):
                self.vfunction_for_K[self.set_into_index(i)] = characteristic_function[i]

#characteristic_function = {(1,):100,(2,):200,(3,):300,(1,2):500,(2,3):600,(1,3):700,(1,2,3):1000}

In [3]:
class CGA:
    def __init__(self, N, k):
        self.N = N
        self.order = k
        self.gen_w_vec()
        self.gen_v_func()

    def gen_w_vec(self):
        self.w_vec = {}
        self.coalitions = []
        for i in range(1, self.order+1):
            for j in list(itertools.combinations(self.N, i)):
                self.w_vec[j] = 0

    def gen_v_func(self):
        self.v_func = {}
        for i in range(1, len(self.N)+1):
            for j in itertools.combinations(self.N, i):
                value = 0
                for m in range(1, self.order+1):
                    if m > len(j):
                        break
                    for n in itertools.combinations(j, m):
                        value = value+self.w_vec[n]
                self.v_func[j] = value

    def update_w_vec(self, w_v_new):
        for i in w_v_new:
            if i in self.w_vec:
                self.w_vec[i] = w_v_new[i]
        self.gen_v_func()

    def show_w_vec(self):
        print(self.w_vec)

    def show_v_func(self):
        print(self.v_func)

    def show_coalitions(self):
        print(self.coalitions)


def harsanyiDividends(N, v_func):
    dividends = {}
    for size in range(1, len(N)+1):
        for coalition in set(itertools.combinations(N, r=size)):
            base, shift = v_func[coalition], 0
            for smallsize in range(1, size):
                for subs in set(itertools.combinations(coalition, r=smallsize)):
                    shift += dividends[subs]
            dividends[coalition] = base - shift
    return dividends


In [4]:
N=["A","B","C"]
v_func={('A',): 1, ('B',): 1, ('C',): 1, ('A', 'B'): 2, ('A', 'C'): 2, ('B', 'C'): 2, ('A', 'B', 'C'): 4}

cga=CGA(N,2)
cga.show_v_func()
cga.show_w_vec()

print(harsanyiDividends(N,v_func))
cga.update_w_vec(harsanyiDividends(N,v_func))
cga.show_v_func()


{('A',): 0, ('B',): 0, ('C',): 0, ('A', 'B'): 0, ('A', 'C'): 0, ('B', 'C'): 0, ('A', 'B', 'C'): 0}
{('A',): 0, ('B',): 0, ('C',): 0, ('A', 'B'): 0, ('A', 'C'): 0, ('B', 'C'): 0}
{('A',): 1, ('B',): 1, ('C',): 1, ('A', 'B'): 0, ('A', 'C'): 0, ('B', 'C'): 0, ('A', 'B', 'C'): 1}
{('A',): 1, ('B',): 1, ('C',): 1, ('A', 'B'): 2, ('A', 'C'): 2, ('B', 'C'): 2, ('A', 'B', 'C'): 3}


In [5]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor


data = [
    [1,0,0],
    [0,1,0],
    [0,0,1],
    [1,1,0],
    [1,0,1],
    [0,1,1],
    [1,1,1]
]
target = [200,100,300,400,500,500,800]
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33)

sgd = SGDRegressor(penalty = 'l2',max_iter=1e9)
sgd.fit(X_train, y_train)
y_pred_sklearn = sgd.predict(X_test)
print(X_test)
print(y_pred_sklearn)

[[0, 0, 1], [0, 1, 1], [1, 1, 0]]
[118.74946255 410.02051676 488.91391124]


In [6]:
print(X_train)
print(y_train)
X_train_new = [
    [1,0,1,0,1,0],
    [0,0,1,0,0,0],
    [1,1,0,1,0,0],
    [1,1,1,1,1,1]
]
X_train_new = np.array(X_train_new).T
print(X_train_new)

[[1, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1]]
[500, 100, 200, 800]
[[1 0 1 1]
 [0 0 1 1]
 [1 1 0 1]
 [0 0 1 1]
 [1 0 0 1]
 [0 0 0 1]]


In [7]:
from scipy.special import comb, perm

def compute_w_vec_size(N,k):
    C = 0
    for i in range(k+1):
        C = C + comb(N,i)
    return C

N=3
k=2
print(compute_w_vec_size(3,2))

7.0


In [55]:
n_iter=100  # number of iterations
r=0.1      # learning rate

N=len(X_train)
cga = CGA([1,2,3],2)
w_BGD = np.array(list(cga.w_vec.values())).reshape(-1,1)
length = len(cga.w_vec.values())
gradient = np.zeros((1,length))

for i in range(6):
    gradient[0][i] = np.random.rand()
             
for j in range(n_iter):
    for i in range(N):
        temp = X_train_new.T[i].reshape(-1,1)
        gradient = gradient + 2 *(np.dot(np.array(w_BGD).T,temp)-y_train[i])* temp.T
    gradient = gradient/N
    w_BGD = (1-r/N)*w_BGD - r*gradient.T
    
# print(w_BGD)
    
new_w_vec={}
for i in range(len(cga.w_vec.keys())):
    new_w_vec[list(cga.w_vec.keys())[i]]=w_BGD[i]
    
cga.update_w_vec(new_w_vec)
y_pred_BGD = cga.v_func
print(y_pred_BGD)
print(target)

{(1,): array([122.84126053]), (2,): array([56.80014475]), (3,): array([132.83535858]), (1, 2): array([236.44195457]), (1, 3): array([475.20628146]), (2, 3): array([343.12613626]), (1, 2, 3): array([742.29760843])}
[200, 100, 300, 400, 500, 500, 800]
